In [107]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset, Reader
from surprise import SVD

np.random.seed(42)

def get_top_n(predictions, n=10):  
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        i = 0
        top_boofer = []
        
        while True:
            if (len(top_boofer) == n) or (i == len(user_ratings)):
                top_n[uid] = top_boofer
                
                break
                
            elif (user_ratings[i] in top_boofer) or (user_ratings[i][0] in list(df[df['UID'] == uid]['JID'])):
                i+=1
                
            else:
                top_boofer.append(user_ratings[i])
                i+=1
            
    return top_n

df = pd.read_csv('train_joke_df.csv')

df = df.sort_values(by=['UID', 'JID'])
df = df.reset_index(drop=True)

reader = Reader(rating_scale=(-10, 10))

data = Dataset.load_from_df(df[['UID', 'JID', 'Rating']], reader)

trainset_data = data.build_full_trainset()

svd = SVD(verbose=False, n_epochs=16, lr_all = 0.002, reg_all = 0.13, random_state=42)
svd.fit(trainset_data)

testset = trainset_data.build_anti_testset()
predictions_svd = svd.test(testset)

top_n = get_top_n(predictions_svd)
users = list(top_n.keys())

to_pred = pd.read_csv('input.csv')
users_pred = list(to_pred['UID'])

df_r = df.drop('UID', axis = 1).groupby(['JID']).mean().sort_values (by = ['Rating'], ascending = False)
basic_top = []
basic_top.append(df_r.iloc[0][0])
boofer = []
for i in range(len(df['JID'].unique())):
    boofer.append(df_r.iloc[i].name)
basic_top.append(boofer)

result = {}

for user in users_pred:
    boofer = []
    if user in users:
        top_1_JID = top_n[user][0][0]
        top_1 = top_n[user][0][1]
        top_10 = []
        for joke in top_n[user]:
            top_10.append(joke[0])
        
        if len(top_10) < 10:
            i = 0
        
        boofer.append(top_1_JID)
        boofer.append(top_1)
        boofer.append(top_10)
        
        
    else:
        top_1 = basic_top[0] 
        top_10 = basic_top[1][:10]
        boofer.append(df_r.iloc[0].name)
        boofer.append(top_1)
        boofer.append(top_10)
    
    i = 0
    while (len(boofer[2]) < 10):
        if i == len(basic_top[1]):
            break
                    
        elif (basic_top[1][i] in df[df['UID'] == user]['JID'].unique()) or (basic_top[1][i] in boofer[2]):
            i+=1
                
        else:
            boofer[2].append(basic_top[1][i])
            i+=1
    
    result[user] = boofer
    
    if len(result[user][2]) < 10:
        for i in range(len(basic_top)):
            if (basic_top[1][i] not in df[df['UID'] == user]['JID'].unique()) and (basic_top[1][i] not in result[user][2]):
                result[user][2].append(basic_top[1][i])
                if len(result[user][2]) == 10:
                    break

df_p = pd.DataFrame.from_dict (result, orient='index').reset_index()
df_p.columns = ['UID', 'Top_1_JID', 'Top_1_Raiting', 'Top_10_Recomendations']
df_p.to_csv('output.csv', index = False)